In [ ]:
!pip install -U sentence-transformers

In [ ]:
import requests
import json

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, util

import requests
import json

sim_model_name="Sakil/sentence_similarity_semantic_search"
sim_model = SentenceTransformer(sim_model_name)

In [ ]:
def sentence_in(reddits):
    sub_rules = []
    sub_out = []
    num_rules = []
    for sub, rules in reddits.items():
        sub_out.append(sub)
        i = 0
        for rule in rules.values():
            sub_rules.append(rule["short_name"] + " " + rule["description"])
            i = i + 1
        num_rules.append(i)
    return sub_rules, sub_out, num_rules

def sub_rules():
    f_open = open("leftrules.json")
    reddits = json.load(f_open)
    data_all = sentence_in(reddits)
    return data_all

data, subs_out, num_rules = sub_rules()

In [ ]:
print(data)
print(subs_out)
print(num_rules)
print(sum(num_rules)/ len(num_rules))

In [ ]:
#Encode all sentences
embeddings = sim_model.encode(data)

#Compute cosine similarity between all pairs
cos_sim = util.cos_sim(embeddings, embeddings)

#Add all pairs to a list with their cosine similarity score
all_sentence_combinations = []

for i in range(len(cos_sim)-1):
    for j in range(i+1, len(cos_sim)):
        if cos_sim[i][j] > 0.9:
            xx = 0
            ii = 0
            jj = 0
            sub_i = " "
            sub_j = " "
            for w in range(len(num_rules)):
                if i >= xx - 1 and i < xx + num_rules[w]:
                    sub_i = subs_out[ii]
                if j >= xx - 1 and j < xx + num_rules[w]:
                    sub_j = subs_out[jj]
                ii += 1
                jj += 1
                xx += num_rules[w]
            if sub_i != sub_j:
                all_sentence_combinations.append([cos_sim[i][j], data[i], data[j], sub_i, sub_j])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

graph_1 = []
graph_count = []
#counter = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.95, 1]
counter = [0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
for n in counter:
    tab = 0
    for i in range(len(cos_sim)-1):
        for j in range(i+1, len(cos_sim)):
            if cos_sim[i][j] > n:
                tab += 1
    graph_count.append(tab)

xpoints = np.array(counter)
ypoints = np.array(graph_count)


plt.plot(xpoints, ypoints)
plt.show()



In [ ]:
print(graph_count)

In [ ]:
print(len(all_sentence_combinations))

In [ ]:
# for row in all_sentence_combinations:
#     print(row)

In [ ]:
all_sentence_combinations_over_no = []
for row in all_sentence_combinations:
    row_new = []
    row_new.append(row[0])
    row_new.append(row[1])
    row_new.append(row[2])
    row_new.append([row[3], row[4]])
    all_sentence_combinations_over_no.append(row_new)

all_sentence_combinations_over_no = sorted(all_sentence_combinations_over_no, key=lambda x: x[3], reverse=True)

for row in all_sentence_combinations_over_no:
    # find rule itself with most overlap to other subreddits
    # community do unique on the rules, one rule can only map once? map to number of rules?


    print(row[3], row)


In [ ]:
import itertools

# def split_list(lst, val):
#     return [list(group) for k,
#             group in
#             itertools.groupby(lst, lambda x: x[3]) if not k]

# original_lst = [1, 4, 5, 6, 4, 5, 6, 5, 4]
# split_sentences = split_list(, 6)

# print("The original list:", original_lst)
# print("The list after splitting by a value:", split_sentences)

In [ ]:
all_sentence_combinations_out = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

#print("Top-5 most similar pairs:")

overlap_A = []
overlap_both = []
for row in all_sentence_combinations_out:
    overlap_A.append(row[3])
    overlap_A.append(row[4])
    overlap_both.append([row[3], row[4]])
print(overlap_A)
print(overlap_both)

In [ ]:
import pandas as pd

# declaring the list

count = pd.Series(overlap_A).value_counts()
print("Element Count")
print(count)

In [ ]:
count = pd.Series(overlap_both).value_counts()
print("Element Count")
print(count)

In [ ]:
make_unique = sorted(all_sentence_combinations_over_no, key=lambda x: x[3], reverse=True)

same = make_unique[0][3]
print(same)


Sentiment

In [ ]:
from transformers import pipeline

classifier_emo = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

In [ ]:
model_outputs = classifier_emo(data)
print(model_outputs)






In [ ]:
emotions = []
high_sub = []
i = 0
for line in model_outputs:
    for emo in line:
        if emo['score'] > 0.5:
            emotions.append(emo['label'])
            if emo['label'] != 'neutral':
                high_sub.append([i, emo['label'], emo['score']])
    i += 1

In [ ]:
count_sentiment = pd.Series(emotions).value_counts()
print("Emotion over 0.5 Count")
print(count_sentiment)

In [ ]:
print(high_sub)

In [ ]:
emotions = []
for line in model_outputs:
    for emo in line:
        if emo['score'] > 0.75:
            print(emo)
            emotions.append(emo['label'])

In [ ]:
count_sentiment = pd.Series(emotions).value_counts()
print("Emotion over 0.75 Count")
print(count_sentiment)

In [ ]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

In [ ]:
cat = sentiment_pipeline(data)

In [ ]:
pos = 0
neg = 0
for line in cat:
  if line['label'] == 'POSITIVE':
      pos += 1
  if line['label'] == 'NEGATIVE':
      neg += 1

print("pos: ", pos)
print("neg: ", neg)

In [ ]:
print("Postive Rules: ", pos)
print("Negative Rules ", neg)
print("Percentage Negative: ", neg/(pos + neg) * 100, "%")